In [1]:
import json

#  Define your manually labeled ground truth entities
ground_truth_entities = [
    {"text": "Ministères sociaux", "type": "ORG"},
    {"text": "Direction des finances, des achats et des services", "type": "ORG"},
    {"text": "14 avenue Duquesne", "type": "LOC"},
    {"text": "75350 Paris 07 SP", "type": "LOC"},
    {"text": "Ministère des Solidarités et de la Santé", "type": "ORG"},
    {"text": "Ministère du Travail, de l Emploi et de l Insertion", "type": "ORG"},
    {"text": "Tour Mirabeau", "type": "LOC"},
    {"text": "Secrétariat général de la défense et de la sécurité nationale", "type": "ORG"},
    {"text": "Clément COQUERY", "type": "PER"},
    {"text": "clement.coquery", "type": "PER"},
    {"text": "beatrice.calvet", "type": "PER"},
    {"text": "DFAS -DADD -RESEAU -ACHAT", "type": "ORG"},
    {"text": "INSEE", "type": "ORG"},
    {"text": "JORF", "type": "ORG"},
    {"text": "la Banque centrale européenne", "type": "ORG"},
    {"text": "MOM, VA et VSR", "type": "ORG"},
    {"text": "Chorus Pro", "type": "ORG"},
    {"text": "helene.merlet", "type": "PER"},
    {"text": "stephanie.smaghe", "type": "PER"}
]

# Save to JSON file
json_filename = "ground_truth_entities.json"
with open(json_filename, "w", encoding="utf-8") as f:
    json.dump(ground_truth_entities, f, indent=4, ensure_ascii=False)

print(f"Ground truth entities saved to {json_filename}!")


Ground truth entities saved to ground_truth_entities.json!


In [ ]:
import json
from seqeval.metrics import classification_report

#  Function to load extracted entities from a JSON file
def load_extracted_entities(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        return json.load(f)

# Function to convert extracted entities into BIO format
def convert_to_bio_format(entity_list, text):
    """Converts named entities into BIO format for evaluation."""
    bio_labels = ["O"] * len(text.split())  # Initialize all tokens as 'O' (outside entity)
    words = text.split()
    
    for entity in entity_list:
        entity_text = entity["text"]
        entity_type = entity["type"]
        entity_words = entity_text.split()
        
        for i, word in enumerate(words):
            if word == entity_words[0]:  # First occurrence of entity
                bio_labels[i] = f"B-{entity_type}"
                for j in range(1, len(entity_words)):
                    if i + j < len(words):
                        bio_labels[i + j] = f"I-{entity_type}"
    
    return bio_labels

#  Load your ground truth (manually annotated entities)
ground_truth_json = "ground_truth_entities.json"  # Replace with your actual file
ground_truth_entities = load_extracted_entities(ground_truth_json)

# ✅ Define your test text (should be the same text used for entity extraction)
test_text_file = "test_contract.txt"
with open(test_text_file, "r", encoding="utf-8") as f:
    test_text = f.read()

# ✅ Convert ground truth entities into BIO format
true_labels = convert_to_bio_format(ground_truth_entities, test_text)

# ✅ List of model output files to evaluate
model_files = {
    "Flair Pretrained": "extracted_entities_flair.json",
    "Flair Fine-Tuned": "flair_finetuned_entities.json",
    "XLM-RoBERTa": "extracted_entities_Roberta.json",
    "CamemBERT": "extracted_entities_Camembert.json"
}

# ✅ Evaluate each model
model_scores = {}

for model_name, json_file in model_files.items():
    print(f"\n🔹 Evaluating {model_name}...")

    # Load model predictions
    predicted_entities = load_extracted_entities(json_file)

    # Convert predictions to BIO format
    predicted_labels = convert_to_bio_format(predicted_entities, test_text)

    # Compute classification report
    report = classification_report([true_labels], [predicted_labels], digits=4)

    # Store scores for later comparison
    model_scores[model_name] = report

    # Print results
    print(report)

# ✅ Compare all models in a structured output
print("\n📊 Final Model Comparison:\n")
for model, report in model_scores.items():
    print(f"\n🔹 {model}:\n{report}")
